In [1]:
import pandas as pd

In [3]:
data = pd.read_excel('data/아파트(매매)_실거래가_20220721131644.xlsx')

C:\Users\medici\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
data['a'] = '서울특별시'
data['addr'] = data['a']+' '+data['자치구 명']
data.drop(['a', '자치구 명', '행정동 명', '공간좌표', '행정동 코드', '자치구 코드'], axis=1, inplace=True)
address = data['addr']

In [4]:
address.head()

0    서울시 강동구
1    서울시 강동구
2    서울시 강동구
3    서울시 강동구
4    서울시 강동구
Name: addr, dtype: object

In [5]:
for i in range(len(address)):
    a = address[i].split(' ')
    address[i] = " ".join(a[0:2])

C:\Users\2005-009\AppData\Local\Temp/ipykernel_2692/3341604468.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address[i] = " ".join(a[0:2])


In [6]:
address

0       서울시 강동구
1       서울시 강동구
2       서울시 강동구
3       서울시 강동구
4       서울시 강동구
         ...   
2115    서울시 종로구
2116    서울시 종로구
2117    서울시 종로구
2118    서울시 종로구
2119    서울시 종로구
Name: addr, Length: 2120, dtype: object

In [7]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

In [8]:
# 위도, 경도 반환하는 함수
def geocoding(address):
    geo = geo_local.geocode(address)
    x_y = [geo.latitude, geo.longitude]
    return x_y

In [9]:
latitude = []
longitude =[]

for i in address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [40]:
address_df = pd.DataFrame({'위도':latitude,'경도':longitude})
addr = pd.concat([data, address_df], axis=1)
addr

,거래년도,소형(60평방미터 이하) 거래건수,소형 거래금액(평균),중소형(60 ~ 85평방미터) 건래건수,중소형 거래금액(평균),중형(85 ~ 102평방미터) 거래건수,중형 거래금액(평균),중대형(102 ~ 135평방미터) 거래건수,중대형 거래금액(평균),대형(135평방미터 이상) 거래건수,대형 거래금액(평균),addr,위도,경도
0,2020,163,651021472.4,313,781737699.7,0,0,59,937338983.1,4,874500000,서울시 강동구,37.530000,127.123700
1,2020,13,1382692308,25,1477200000,44,1634693182,0,0,0,0,서울시 강동구,37.530000,127.123700
2,2020,400,258322500,375,597910666.7,17,758382352.9,46,913717391.3,11,880000000,서울시 강동구,37.530000,127.123700
3,2020,37,497864864.9,125,582233600,9,609722222.2,13,635730769.2,3,1050000000,서울시 강동구,37.530000,127.123700
4,2020,64,414906250,46,686195652.2,2,805000000,10,774800000,0,0,서울시 강동구,37.530000,127.123700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2115,2014,1,300000000,2,385000000,1,270000000,0,0,0,0,서울시 종로구,37.580695,126.982799
2116,2014,29,374693103.4,117,527682906,0,0,35,648678571.4,2,992000000,서울시 종로구,37.580695,126.982799
2117,2014,31,208129032.3,19,375526315.8,7,471714285.7,3,682666666.7,70,1367007234,서울시 종로구,37.580695,126.982799
2118,2014,5,243200000,4,343250000,0,0,2,485000000,0,0,서울시 종로구,37.580695,126.982799


In [22]:
import folium
from haversine import haversine, Unit
from geopy.geocoders import Nominatim

import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [41]:
kdg = addr[addr['addr']=='서울시 강동구']
kdg

,거래년도,소형(60평방미터 이하) 거래건수,소형 거래금액(평균),중소형(60 ~ 85평방미터) 건래건수,중소형 거래금액(평균),중형(85 ~ 102평방미터) 거래건수,중형 거래금액(평균),중대형(102 ~ 135평방미터) 거래건수,중대형 거래금액(평균),대형(135평방미터 이상) 거래건수,대형 거래금액(평균),addr,위도,경도
0,2020,163,651021472.4,313,781737699.7,0,0,59,937338983.1,4,874500000,서울시 강동구,37.53,127.1237
1,2020,13,1382692308,25,1477200000,44,1634693182,0,0,0,0,서울시 강동구,37.53,127.1237
2,2020,400,258322500,375,597910666.7,17,758382352.9,46,913717391.3,11,880000000,서울시 강동구,37.53,127.1237
3,2020,37,497864864.9,125,582233600,9,609722222.2,13,635730769.2,3,1050000000,서울시 강동구,37.53,127.1237
4,2020,64,414906250,46,686195652.2,2,805000000,10,774800000,0,0,서울시 강동구,37.53,127.1237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,2014,210,403100419,126,497067539.7,7,433714285.7,22,764681818.2,41,1006219512,서울시 강동구,37.53,127.1237
1711,2014,0,0,216,444236111.1,0,0,121,594876033.1,0,0,서울시 강동구,37.53,127.1237
1712,2014,262,299004389.3,388,397877061.9,18,507944444.4,45,602877777.8,1,598000000,서울시 강동구,37.53,127.1237
1713,2014,257,383794941.6,97,460194295,0,0,0,0,0,0,서울시 강동구,37.53,127.1237


In [44]:
map = folium.Map(location=[37.578608, 126.798153], zoom_start=10)

for i in range(len(kdg)):
    folium.Marker([kdg.iloc[i,12:13], kdg.iloc[i,13:14]],popup=kdg.iloc[i,1:10] ,icon=folium.Icon(icon='building', color='red', prefix='fa')).add_to(map)
map

In [16]:
stay = pd.read_csv('data/서울교통공사_1-8호선 지하철역 위경도 좌표정보_20220621.csv', encoding='cp949')
stay.index = stay['역명']
del stay['역명']
stay

,연번,호선,역번호,경도,위도,데이터기준일자
역명,,,,,,
서울,1,1,150,126.972533,37.553150,1974-02-28
시청,2,1,151,126.975407,37.563590,1974-08-15
종각,3,1,152,126.983116,37.570203,1974-08-15
종로3가,4,1,153,126.992095,37.570429,1974-08-15
종로5가,5,1,154,127.001900,37.570971,1974-03-31
...,...,...,...,...,...,...
남한산성입구,272,8,2823,127.159845,37.451568,1996-10-31
단대오거리,273,8,2824,127.156735,37.445057,1996-12-28
신흥,274,8,2825,127.147590,37.440952,1996-12-28


In [36]:
kdg.iloc[:,1:2]

,위도
0,37.53
1,37.53
2,37.53
3,37.53
4,37.53
...,...
1710,37.53
1711,37.53
1712,37.53
1713,37.53
